In [ ]:
%matplotlib inline
import textblob
import nltk
import pandas
import Markov

In [ ]:
nltk.download()

# Approach 1: Use an off-the-shelf tagger

For this we're going to use the `TextBlob` library, as we did in [The Grammar of Truth and Lies](https://youtu.be/OyA59kIQcAU)

In [ ]:
text = """Peter Bleackley is a self-employed data scientist and computational linguist. He has undertaken 
Research and Development projects for clients ranging from startups to multinationals. He will shortly be 
starting a webinar series entitled "Ask a Data Scientist". """

blob = textblob.blob.TextBlob(text)

[sentence.pos_tags for sentence in blob.sentences]